Instalación de la libreia deap, si requiere mayor información consultar:

https://deap.readthedocs.io/en/master/

Integrantes:
Adrian Felipe Gironza - Arlex Fabian Galindez


In [ ]:
!pip install deap

In [ ]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import numpy as np
import json
from collections import Counter

In [ ]:
def cargar_datos_asignaturas():
    # Cargar los datos de asignaturas desde un archivo JSON
    with open("asignaturas.json", encoding="utf-8-sig") as file:
        datos = json.load(file)
    return datos

def cargar_datos_salones():
    # Cargar los datos de salones desde un archivo JSON
    with open("salones.json", encoding="utf-8-sig") as file:
        datos_sal = json.load(file)
    return datos_sal

datos_asignaturas = cargar_datos_asignaturas()
datos_salones = cargar_datos_salones()

cantasignaturas = len(datos_asignaturas)
cantdias = 5
cantfranjas=6
cantsalones= len(datos_salones)

In [ ]:
def initIndividual(icls):
    individuo = np.random.randint(cantasignaturas, size=(cantdias, cantfranjas, cantsalones))
    return icls(individuo)

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual_guess)

EVALUACION

In [ ]:
def evalOneMax(individual):
    fitness = 0
    fitness += 100*verificar_asignaturas_repetidas(individual)
    fitness += 10*validar_profesor(individual)
    fitness += verificar_semestres_misma_franja(individual)
    fitness += validar_profesor_nn(individual)
    fitness += validar_franjas_asignatura(individual)
    fitness += validar_tipo_asignatura(individual)
    fitness += 10*validar_asignaturas_asignadas(individual)
    fitness += 10*validar_mas_3_repeticiones(individual)
    #print("fitness",fitness)
    return (fitness,)

def verificar_asignaturas_repetidas(individuo):
    contador_repetidas = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
              #print(individuo_array)
              auxsalon = individuo[dia, franja]  # Acceder a los elementos del individuo
              frecuencias = Counter(auxsalon)
              elementos_repetidos = [elemento for elemento, frecuencia in frecuencias.items() if frecuencia > 1]
              contador_repetidas += len(elementos_repetidos)
    return contador_repetidas

def validar_profesor(individuo):
    contador = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            salones = individuo[dia, franja]  # Obtener los salones asignados en la franja horaria
            profesores = [datos_asignaturas[asignatura]['PROFESOR'] for asignatura in salones]
            frecuencias = Counter(profesores)
            profesores_repetidos = [profesor for profesor, frecuencia in frecuencias.items() if frecuencia > 1]
            contador += len(profesores_repetidos)
    return contador

def validar_profesor_nn(individuo):
    contador = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            asignaturas = individuo[dia, franja, :]  # Obtener las asignaturas de la franja
            profesores = [datos_asignaturas[asignatura]["PROFESOR"] for asignatura in asignaturas]
            contador_nn = sum(1 for p in profesores if p in ["NN3", "Becario", "Asistente de Docencia"])
            if contador_nn > 1:
                contador += 1
    return contador


def verificar_semestres_misma_franja(individuo):
    contador_conflictos = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            asignaturas_franja = individuo[dia, franja]  # Obtener las asignaturas de la franja
            semestres = set()
            for asignatura in asignaturas_franja:
                if asignatura != -1:  # Ignorar las asignaturas no asignadas
                    semestre = datos_asignaturas[asignatura]['ASIGNATURA']
                    if semestre in semestres:
                        contador_conflictos += 1
                        break  # Encontró conflicto, pasa a la siguiente franja
                    semestres.add(semestre)
    return contador_conflictos

def validar_franjas_asignatura(individuo):
    contador_repetidas = 0
    for dia in range(cantdias):
        for asignatura in range(cantasignaturas):
            franjas_dia = individuo[dia, :, :].flatten()  # Obtener todas las franjas del día
            frecuencias = Counter(franjas_dia)
            asignatura_repetida = [frecuencia for _, frecuencia in frecuencias.items() if frecuencia > 1]
            if asignatura_repetida:
                contador_repetidas += 1
    return contador_repetidas

def validar_tipo_asignatura(individuo):
    contador = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):
                codigo_asignatura = individuo[dia, franja, salon]
                asignatura = datos_asignaturas[codigo_asignatura]
                tipo_asignatura = asignatura['TIPOAS']
                tipo_salon = datos_salones[salon]['TIPO']
                if tipo_asignatura == 'T' and tipo_salon != 'TEORICO':
                    contador += 1
                if tipo_asignatura == 'P' and tipo_salon != 'LAB':
                    contador += 1
                if tipo_asignatura == 'TP' and tipo_salon != 'LAB':
                    contador += 1
                if tipo_asignatura == 'P' and int(asignatura['ASIGNATURA']) >= 7 and datos_salones[salon]['CODIGO'] != 'Sala 4':
                    contador += 1
    return contador

def validar_asignaturas_asignadas(individuo):
    conteo_asignaturas = Counter(individuo.flatten())
    contador=0
    for asignatura in datos_asignaturas:
        codigo_asignatura = asignatura['CODIGO']
        jornada_asignatura = asignatura['JORNEC']
        if conteo_asignaturas[codigo_asignatura] != jornada_asignatura:
            contador += abs(conteo_asignaturas[codigo_asignatura] - jornada_asignatura)
    return contador

def validar_mas_3_repeticiones(individual):
    fitness=0
    counts = Counter(individual.flatten())
    for count in counts.values():
        if count > 3:
            fitness += 1
    return fitness

CRUCE

In [ ]:
def cxTwoPointCopy(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoints = sorted(random.sample(range(1, size), 3))
    for i in range(len(cxpoints) // 2):
        cxpoint1, cxpoint2 = cxpoints[i * 2], cxpoints[i * 2 + 1]
        ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
    return ind1, ind2

def cxOnePointCopy(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size-1)  # Punto de cruce aleatorio
    # Realizar el intercambio de partes entre los padres
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:].copy(), ind1[cxpoint:].copy()
    return ind1, ind2

MUTACION

In [ ]:
def mutFlipBit(individual, indpb):
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):
                if random.random() < indpb:
                    individual[dia, franja, salon] = random.randint(0, cantasignaturas-1)
    return individual,

In [ ]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(64)

    pop = toolbox.population(n=1000)
    hof = tools.HallOfFame(1, similar=np.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)


    algorithms.eaSimple(pop, toolbox, cxpb=0.5,mutpb=0.1, ngen=1000, stats=stats,halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
    p,s,h=main()
    print(h[0])

gen	nevals	avg    	std    	min 
0  	1000  	7313.68	535.095	5665
1  	544   	6864.93	426.518	5603
2  	519   	6524.68	356.312	5579
3  	549   	6228.62	310.175	5400
4  	553   	5996.31	277.272	5103
5  	540   	5806.55	227.503	5236
6  	543   	5665.14	227.831	5016
7  	569   	5537.45	207.221	5058
8  	543   	5413.25	195.072	4920
9  	581   	5317.62	216.36 	4620
10 	574   	5210.36	208.485	4701
11 	549   	5115.61	222.548	4561
12 	564   	5021.1 	208.53 	4574
13 	546   	4939.51	236.022	4469
14 	547   	4834.04	220.391	4403
15 	552   	4740.13	209.58 	4377
16 	534   	4673.28	205.921	4336
17 	558   	4619.99	215.831	4336
18 	536   	4560.78	220.173	4284
19 	546   	4507.21	210.486	4269
20 	546   	4462.17	203.225	4143
21 	589   	4434.6 	230.344	4053
22 	580   	4383.59	217.41 	4053
23 	544   	4326.97	205.697	4053
24 	571   	4267.57	226.097	4053
25 	561   	4199.98	227.008	4053
26 	546   	4160   	220.929	4053
27 	549   	4152.06	239.228	4031
28 	564   	4144.3 	241.912	3947
29 	540   	4127.34	231.999	3939
30 	566 